## Imports

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

## Model Class

In [2]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE,self).__init__()
        
        
        self.common_fc = nn.Sequential(
            nn.Linear(28*28, out_features=196), nn.Tanh(),
            nn.Linear(196, out_features=48), nn.Tanh(),
        )
        
        self.mean_fc = nn.Sequential(
            nn.Linear(48, out_features=16), nn.Tanh(),
            nn.Linear(16, out_features=2), nn.Tanh()
        )
        
        self.log_var_fc = nn.Sequential(
            nn.Linear(48, out_features=16), nn.Tanh(),
            nn.Linear(16, out_features=2), nn.Tanh()
        )
        
        self.decoder_fcs = nn.Sequential(
            nn.Linear(2, out_features=16), nn.Tanh(), 
            nn.Linear(16, out_features=48), nn.Tanh(),
            nn.Linear(48, out_features=196), nn.Tanh(),
            nn.Linear(196, out_features=28*28), nn.Tanh(),
        )
        

    def encode(self,x):
        # B,C,H,W
        out = self.common_fc(torch.flatten(x, start_dim=-1))
        mean = self.mean_fc(out)
        log_var = self.log_var_fc(out)
        return mean, log_var


    def sample(self, mean, log_var):
        std = torch.exp(0.5*torch.flatten(log_var, start_dim=-1))
        z = torch.randn_like(torch.flatten(std, start_dim=-1))
        return z * std + mean
    
    
    def decode(self, z):
        flat_z = torch.flatten(z, start_dim=-1)
        out = self.decoder_fcs(flat_z)
        out = torch.reshape(out, [1, 28*28])
        return out
        
    
    def forward(self, batch_x):
        #B,C,H,W
        outputs = []
        for sample in batch_x:
            #Encoder
            mean, log_var = self.encode(torch.flatten(sample))
            #Sampling
            z = self.sample(mean,log_var)
            #Decoder
            outputs.append(self.decode(z))

        out = torch.stack(outputs, dim=1)
        return mean, log_var, out


## Training Parameters

In [3]:
# Device init
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


LEARN_RATE = 1e-3

## Data Creation

In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as transforms

def create_data():
    # ---INITIALIZE DATASET ---
    #Convert pilimage dataset to a standart numpy dataset
    dataset = MNIST(
        root='./data',
        download=True,  # Add this to download the dataset if needed
        transform= transforms.ToTensor()
    )

    TRIM_LEN = int(50_000)  # 60,000 - 50,000 = 10,000 SAMPLES
    TRAIN_PORTION = 0.9 # 90% training 10% everything else
    TRAIN_LEN = int((len(dataset) - TRIM_LEN) * TRAIN_PORTION)
    
    # ---SPLIT DATASET---
    train_ds, test_ds, _ = random_split(
        dataset,  # Split the dataset, not the dataloader!
        [TRAIN_LEN, len(dataset) - TRIM_LEN - TRAIN_LEN, TRIM_LEN]
    )
    #print(f"train length: {len(train_ds)} test_length: {len(test_ds)}")
    
    # ---CREATE DATALOADERS from the split datasets---
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)
    
    return train_loader, test_loader

## Training

In [5]:
def train_vae(model, train_loader, optimizer):
    # Method 1: One-liner
    #single_batch = next(iter(train_loader))[0].to(device)

    for single_batch in train_loader:
        single_batch = single_batch[0].to(device)
        # ---------Feed Forward---------
        # Extract just the generated images for now
        _,_,img_gen_batch = model.forward(single_batch)


        #---------Back Prop---------
        # Loss is calculated by the batch's mean
        loss = 0
        kl_loss = torch.nn.KLDivLoss(reduce='batchmean')
        
        flat_sample = torch.flatten(single_batch,start_dim=1)
        loss = kl_loss(F.log_softmax(img_gen_batch),
                        flat_sample)
        
        optimizer.zero_grad()
        print(loss)
        loss.backward()
        optimizer.step()
        

In [6]:
# Initialize the model
my_vae = VAE().to(device)
optim = torch.optim.Adam(params=my_vae.parameters(),lr = LEARN_RATE)


train_loader,test_loader = create_data()
train_vae(model=my_vae, train_loader=train_loader,
          optimizer=optim)

C:\Users\orian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
C:\Users\orian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
C:\Users\orian\AppData\Local\Temp\ipykernel_42676\2710401959.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  loss = kl_loss(F.log_softmax(img_gen_batch),
C:\Users\orian\AppData\Local\P

tensor(-0.0297, grad_fn=<MeanBackward0>)
tensor(-0.0284, grad_fn=<MeanBackward0>)
tensor(-0.0257, grad_fn=<MeanBackward0>)
tensor(-0.0278, grad_fn=<MeanBackward0>)
tensor(-0.0280, grad_fn=<MeanBackward0>)
tensor(-0.0285, grad_fn=<MeanBackward0>)
tensor(-0.0275, grad_fn=<MeanBackward0>)
tensor(-0.0276, grad_fn=<MeanBackward0>)
tensor(-0.0318, grad_fn=<MeanBackward0>)
tensor(-0.0254, grad_fn=<MeanBackward0>)
tensor(-0.0274, grad_fn=<MeanBackward0>)
tensor(-0.0282, grad_fn=<MeanBackward0>)
tensor(-0.0298, grad_fn=<MeanBackward0>)
tensor(-0.0271, grad_fn=<MeanBackward0>)
tensor(-0.0269, grad_fn=<MeanBackward0>)
tensor(-0.0298, grad_fn=<MeanBackward0>)
tensor(-0.0294, grad_fn=<MeanBackward0>)
tensor(-0.0294, grad_fn=<MeanBackward0>)
tensor(-0.0255, grad_fn=<MeanBackward0>)
tensor(-0.0264, grad_fn=<MeanBackward0>)
tensor(-0.0302, grad_fn=<MeanBackward0>)
tensor(-0.0263, grad_fn=<MeanBackward0>)
tensor(-0.0275, grad_fn=<MeanBackward0>)
tensor(-0.0294, grad_fn=<MeanBackward0>)
tensor(-0.0279, 